# 그루핑

1. \() : 패턴을 그룹화
2. group(인덱스)
3. 그루핑된 문자열 재참조하기
4. 그루핑된 문자열에 이름 붙이기


In [4]:
str = "ABCABCABC"


In [5]:
import re

In [6]:
p = re.compile(f"(ABC){1,3}")
m = p.search(str)
m

In [14]:
str = """이이름 010-1000-1000
김이름 010-2000-2000
Rachel 010-3000-3000
"""

In [17]:
# p = re.compile(r"[a-zㄱ-ㅎ가-힣]+\s+010\D?\d{4}\D?\d{4}$", re.I|re.M)
p = re.compile(r"([a-zㄱ-ㅎ가-힣]+)\s+(010\D?\d{4}\D?\d{4}$)", re.I|re.M)

In [18]:
m = p.findall(str)
m

[('이이름', '010-1000-1000'),
 ('김이름', '010-2000-2000'),
 ('Rachel', '010-3000-3000')]

In [19]:
for m in p.finditer(str):
    print(f"m.group():{m.group()}")
    print(f"m.group(1):{m.group(1)}")
    print(f"m.group(2):{m.group(2)}")

m.group():이이름 010-1000-1000
m.group(1):이이름
m.group(2):010-1000-1000
m.group():김이름 010-2000-2000
m.group(1):김이름
m.group(2):010-2000-2000
m.group():Rachel 010-3000-3000
m.group(1):Rachel
m.group(2):010-3000-3000


## 문자열 바꾸기

1. sub
    - count : 지정하지 않으면 전체 문자열에서 모든 패턴을 치환, 숫자를 지정된 숫자만큼의 패턴만 치환
2. subn
    - sub + count 기능 포함, 반환값은 튜플

In [20]:
str = """이이름 010-1000-1000
김이름 010-2000-2000
Rachel 010-3000-3000
"""

In [21]:
p = re.compile(r"([a-zㄱ-ㅎ가-힣]+)\s+(010\D?\d{4}\D?\d{4}$)", re.I|re.M)

In [23]:
str2 = p.sub(r"\g<2> \g<1>", str)
print(str2)

010-1000-1000 이이름
010-2000-2000 김이름
010-3000-3000 Rachel



- 그룹핑의 이름을 설정

```
(?P<이름>)
```


In [26]:
p = re.compile(r"(?P<name>^[a-zㄱ-ㅎ가-힣]+)\s+(?P<mobile>010\D?\d{4}\D?\d{4}$)", re.I|re.M)

In [29]:
str2 = p.sub(r"\g<mobile> \g<name>", str)
print(str2)

010-1000-1000 이이름
010-2000-2000 김이름
010-3000-3000 Rachel



- 그룹화된 패턴을 재활용

In [31]:
str = "Paris in the the spring"

In [37]:
#p  = re.compile(r"(\w+)\s+\1")
p = re.compile(r"(?P<word>\w+)\s+(?P=word)")

In [38]:
m = p.search(str)
m

<re.Match object; span=(9, 16), match='the the'>

In [39]:
str = "one little, two little, three little indians"

In [40]:
p = re.compile("little")

In [42]:
# str2 = p.sub("big",str)
str2 = p.sub("big",str, count=2) # 왼쪽부터 매칭되는 2개만 치환
str2

'one big, two big, three little indians'

In [46]:
result = p.subn("big",str, 2)
result

('one big, two big, three little indians', 2)

In [47]:
str2 = p.sub(lambda m: m.group().upper(), str)
str2

'one LITTLE, two LITTLE, three LITTLE indians'

### 전방 탐색
1. 긍정형 전방 탐색
- 특정 패턴의 앞쪽 패턴으로 한정
```
앞쪽패턴(?=특정패턴)
```
2. 부정형 전방 탐색
- 특정 패턴이 아닌 앞쪽 패턴으로 한정
```
앞쪽패턴(?!특정패턴)
```

In [48]:
str = """https://www.naver.com
https://www.daum.net
http://www.webnmobile.net
"""

In [51]:
# p = re.compile(r"http[s]?://",re.M)
p = re.compile(r"http[s]?(?=://)")

In [52]:
items = p.findall(str)
items

['https', 'https', 'http']

In [66]:
p = re.compile(r"http[s]?://(www\.)?([^.]+(?=\.net$))", re.M)

In [67]:
for item in p.finditer(str):
    print(item.group(2))

daum
webnmobile


In [68]:
p = re.compile(r"https[s]?://(www\.)?([^.]+(?!\.net$))", re.M) # 전방 부정 탐색, .net으로 끝나지 않는 앞쪽 패턴 문자

In [69]:
print(str)
for item in p.finditer(str):
    print(item.group(2))

https://www.naver.com
https://www.daum.net
http://www.webnmobile.net

naver
dau


### greedy와 non-greedy
- 패턴+, 패턴*, 패턴?, 패턴{n,m} : 최대 매칭(greedy)
- 패턴+?, 패턴*?, 패턴??, 패턴{n,m}? : 최소 매칭(non-greedy)

In [70]:
str = "<html><head><title>사이트 제목</title></head><body></body></html>"

In [71]:
p = re.compile(r"<.*>") # greedy : 최대 매칭

In [72]:
m = p.search(str)
m.group()

'<html><head><title>사이트 제목</title></head><body></body></html>'

In [73]:
p = re.compile(r"<.*?>") # non-greedy : 최소 매칭
m = p.search(str)
m.group()

'<html>'

In [74]:
import requests

In [75]:
response = requests.get("https://finance.naver.com/marketindex/exchangeList.naver")

In [89]:
data = response.text.split(r'<td class="tit">')[2]
data

'<a href="/marketindex/exchangeDetail.naver?marketindexCd=FX_EURKRW" target="_parent" onclick="parent.clickcr(this, \'exl.exlist\', \'FX_EURKRW\', \'2\', event);">\n\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t유럽연합 EUR\n\t\t\t\t\n\t\t\t\t</a></td>\n\t\t\t\t<td class="sale">1,564.04</td>\n\t\t\t\t<td>\n\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t1,595.16\n\t\t\t\t\t\n\t\t\t\t</td>\n\t\t\t\t<td>\n\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t1,532.92\n\t\t\t\t\t\n\t\t\t\t</td>\n\t\t\t\t<td>\n\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t1,579.68\n\t\t\t\t\t\n\t\t\t\t</td>\n\t\t\t\t<td>\n\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t1,548.40\n\t\t\t\t\t\n\t\t\t\t</td>\n\t\t\t\t<td>1.138</td>\n\t\t\t\t</tr>\n\t\t\t\t\n\t\t\t\t<tr>\n\t\t\t\t'